<a href="https://colab.research.google.com/github/alexlautw9527/for_notebook/blob/main/%E9%89%85%E4%BA%A8%E7%B6%B2%E6%96%87%E5%AD%97%E6%8E%A2%E5%8B%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install transformers
!pip install gdown
!pip install opencc-python-reimplemented
!pip install googletrans==3.1.0a0
!pip install fake_useragent

In [114]:
def save():
  %notebook 鉅亨網文字探勘.ipynb 
  !cp 鉅亨網文字探勘.ipynb "/content/drive/MyDrive/Colab Notebooks"

In [115]:
save()

In [2]:
import torch
import numpy as np
import pandas as pd

In [3]:
from transformers import BertModel, BertTokenizer
PATH = '/content/FinBERT_L-12_H-768_A-12_pytorch'

In [4]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent


In [5]:
import time
import datetime as dt

import time
import tqdm

In [6]:
from opencc import OpenCC
cc = OpenCC('t2s')

In [ ]:
#!gdown https://drive.google.com/uc?id=1qW1YWtw3q9Q28QThrIY-rDU9Gl-SLIKO
#!unzip /content/FinBERT_L-12_H-768_A-12_pytorch.zip

# 財金BERT

In [ ]:
tokenizer = BertTokenizer.from_pretrained(PATH ,local_files_only=True)
model = BertModel.from_pretrained(PATH ,local_files_only=True)

In [ ]:
sector_dict = {
'健護': '醫療保健服務類股',
'科技': '科技類股', 
'公用事業': '公用事業類股',
'生技':'生技類股', 
'能源':'能源類股',
'替代能源':'替代能源類股',
'消費':'消費品類股',
'金融':'金融類股', 
'礦業':'礦業',
'天然資源':'天然資源', 
'貴金屬現貨':'貴金屬',
'農產品':'農產品',
'貴金屬類股':'貴金屬',
}

In [ ]:
mkt_dict= [
'新興市場債',
'高收益債',
'全球債',
'投資等級公司債',
'美國公債',
'亞債',
'REITs不動產投資信託',
'新臺幣類貨幣',
'外幣類貨幣',
'亞太股市', '美國股市', '全球新興市場', 
'全球型股市', '大中華股市', '中國A股', '印度股市','歐洲股市', 
'東南亞股市', '拉丁美洲股市', 
'韓國股市', '新興歐洲股市', '德國股市', '台灣股市','俄羅斯股市',
'中國H股', '日本股市', '英國股市','泰國股市',
'香港股市', '澳洲股市', '印尼股市', '巴西股市', '歐非中東股市'
]


In [ ]:
mkt_dict = {x:x for x in mkt_dict}
mkt_dict['亞債'] = '亞洲債券'
mkt_dict['REITs不動產投資信託'] = '不動產'

In [ ]:
total_sec_dict = {**sector_dict, **mkt_dict}

In [ ]:
sentences = list(set(total_sec_dict.values()))
sentences = [cc.convert(x) for x in sentences]

In [ ]:
inputs = tokenizer(sentences, padding=True, return_tensors="pt")


In [ ]:
## 單純forward pass 沒要計算梯度
with torch.no_grad():
  outputs = model(**inputs)

In [ ]:
x = outputs['last_hidden_state']
np.savetxt('bert_mkt_emb.tsv', x[:,0].cpu().detach().numpy(), delimiter="\t")
pd.DataFrame(sentences).to_csv('bert_mkt_emb_meta.tsv', header=False, index=False, sep="\t")

# 鉅亨網爬蟲 初步整理


In [ ]:
res = requests.get('https://news.cnyes.com/news/cat/wd_stock')

In [ ]:
soup = BeautifulSoup(res.text, 'html.parser')

In [ ]:
%%capture

"""
news_list = soup.find_all("a", attrs={"class":"_1Zdp"})
[x['href'] for x in news_list]
[x.find('h3').get_text(strip=True) for x in news_list
"""
## .get_text(strip=True) 可避免不必用空白符亂碼

In [ ]:
ua = UserAgent()
headers = {'user-agent': ua.random}

In [ ]:
## 將資料起迄日轉換unix timestamp

start_unix_timestamp = int(time.mktime(dt.date(2021,1,1).timetuple()))
end_unix_timestamp = int(time.mktime(dt.date(2021,11,15).timetuple()))

In [ ]:
url_api = 'https://api.cnyes.com/media/api/v1/newslist/category/tw_stock?startAt={}&endAt={}&limit=30&page={}'.format(start_unix_timestamp, end_unix_timestamp, 1)
res = requests.get(url_api, headers=headers)
res_json = res.json()

In [ ]:
last_page = res_json['items']['last_page']
last_page

In [ ]:
news_data_list = []

for i in tqdm.tqdm(range(1,last_page+1)):
  url_api = 'https://api.cnyes.com/media/api/v1/newslist/category/tw_stock?startAt={}&endAt={}&limit=30&page={}'.format(start_unix_timestamp, end_unix_timestamp, i)
  res = requests.get(url_api, headers=headers)
  res_json = res.json()
  news_data_list.append(res_json)
  time.sleep(1)

100%|██████████| 613/613 [23:20<00:00,  2.29s/it]


In [11]:
import pickle
'''
with open('鉅亨網2021新聞.pickle', 'wb') as f:
  pickle.dump(news_data_list, f)
'''

"\nwith open('鉅亨網2021新聞.pickle', 'wb') as f:\n  pickle.dump(news_data_list, f)\n"

In [12]:
!cp /content/drive/MyDrive/鉅亨網2021新聞.pickle /content

In [13]:
with open('鉅亨網2021新聞.pickle', 'rb') as f:
  news_data_list = pickle.load(f)

In [14]:
import re

In [32]:
## 只保留英文中文數字 

def remove_punctuation(line):
  #rule = re.compile("[^a-z A-Z \u4e00-\u9fa5]")
  rule = re.compile("[^a-z A-Z 0-9 \u4e00-\u9fa5]")
  line = rule.sub('',line)
  return line

def remove_char(line):
  rule = re.compile('&lt;.+?(&gt;|>)')
  line = rule.sub('',line)
  return line


In [16]:
from datetime import datetime
ts = int('1636969777')

# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
print(dt.datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

2021-11-15 09:49:37


In [17]:
## todo:取出該新聞的相關標的

In [148]:
news_extracted_list=[]

for chunk in news_data_list:
  article_content = chunk['items']['data']
  data_dict = [{
      'id':x['newsId'], 
      'title':x['title'], 
      'summary':x['summary'],  
      'article':remove_char(x['content']),  
      #'stock':x['market'],

      'unix_timestamp':x['publishAt'],
      'datetime':dt.datetime.utcfromtimestamp(x['publishAt']).strftime('%Y-%m-%d')
      } for x in article_content
  ]
  news_extracted_list.extend(data_dict)


In [149]:
org_corpus_df = pd.DataFrame(news_extracted_list)

In [150]:
org_corpus_df['article'] = org_corpus_df['article'].str.replace('\n\n',"")
org_corpus_df['article'] = org_corpus_df['article'].str.replace(" ", "")

In [151]:
org_corpus_df['article'] = org_corpus_df['article'].str.replace(" ", "")

In [152]:
org_corpus_df['summary'] = org_corpus_df['summary'].astype(str).apply(lambda x: remove_punctuation(x))

# 分詞 資料整理

In [153]:
text = org_corpus_df['summary'].astype(str).tolist()

In [ ]:
#re.sub(r"(?m)^\s+", "", " ")

### 停用詞

In [127]:
!wget https://raw.githubusercontent.com/goto456/stopwords/master/cn_stopwords.txt

--2021-11-17 07:49:53--  https://raw.githubusercontent.com/goto456/stopwords/master/cn_stopwords.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4717 (4.6K) [text/plain]
Saving to: ‘cn_stopwords.txt’

cn_stopwords.txt    100%[===================>]   4.61K  --.-KB/s    in 0s      

2021-11-17 07:49:54 (67.1 MB/s) - ‘cn_stopwords.txt’ saved [4717/4717]



In [132]:
with open('/content/cn_stopwords.txt', 'r') as f:
  stop_words = f.read().splitlines()

In [134]:
from opencc import OpenCC
cc = OpenCC('s2t')

stop_words = [cc.convert(text) for text in stop_words]

### bert版斷詞 偏慢

In [ ]:
!pip install -U ckip-transformers
#https://github.com/ckiplab/ckip-transformers
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker


In [ ]:
ws_driver = CkipWordSegmenter(level=3,device=0)


In [154]:
corpus_ws_org  = ws_driver(text)


Inference: 100%|██████████| 66/66 [01:00<00:00,  1.10it/s]


### lstm斷詞

In [ ]:
!pip install -U ckiptagger[gdown]

In [24]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

In [25]:
data_utils.download_data_url("./")

In [39]:
ws = WS("./data", disable_cuda=False)

/usr/local/lib/python3.7/dist-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/usr/local/lib/python3.7/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:988: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  partitioner=maybe_partitioner)
/usr/local/lib/python3.7/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:996: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=initializer)


In [40]:
%%time
corpus_ws_org  = ws(text)

CPU times: user 32.9 s, sys: 4.8 s, total: 37.7 s
Wall time: 21.9 s


### 方便套件套件處理的空白分隔模式

In [156]:
## 刪除空白的element
corpus_ws_org = [list(filter(lambda x: x!="" and x!=" ", corpus)) for corpus in corpus_ws_org]


In [158]:
corpus_ws_org
corpus_ws_rm =[]
for corpus in corpus_ws_org:
  pattern = re.compile(r'\s+')
  corpus_ws_rm.append([re.sub(pattern, '', x) for x in corpus])


In [159]:
## 停用詞
corpus_ws_rm = [[x for x in sentence if x not in stop_words] for sentence in corpus_ws_rm]

In [160]:
corpus_white_join = [" ".join(x) for x in corpus_ws_rm]

In [162]:
corpus_white_join[4:10]

['22 家 壽險 前 季 合賺 3389 億 元 破紀錄 淨值比 all pass',
 '',
 '',
 '康控 KY4943 TW 15 日 公告 第三 季 財報 銷貨 客戶 存在 產品 品質 責任 歸屬 認知 差異 提列 263億 元 預期 信用 減損 導致 單季 稅 虧損 來到 2738 億 元 股 虧損 1749 元',
 '台灣 光罩 2338 TW 15 日 公告 為 因應 光罩 業務 發展 組織 擴展 計畫 增設 營運 副總 漢磊 3707 TW 前 營運 副總 曾哲斌 出任 漢磊 長期 著墨 化合物 半導體 領域 光罩 次 派任 具 化合物 半導體 背景 經理人 擔任 要職',
 '']

# text mining

In [163]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

## 中文TF-IDF

`token_pattern='\\b\\w+\\b'` : 讓TF-IDF認得單字詞

`lowercase=False`

In [165]:
from sklearn.feature_extraction.text import CountVectorizer
# step 1
vectoerizer = CountVectorizer(min_df=5, token_pattern='\\b\\w+\\b', lowercase=False)

In [166]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(min_df=5, token_pattern='\\b\\w+\\b', lowercase=False)


In [167]:
tfidf.fit(corpus_white_join)

TfidfVectorizer(lowercase=False, min_df=5, token_pattern='\\b\\w+\\b')

In [168]:
response = tfidf.transform(corpus_white_join)
feature_names = tfidf.get_feature_names_out()

In [169]:
feature_array = np.array(tfidf.get_feature_names_out())
tfidf_sorting = np.argsort(response.toarray()).flatten()[::-1]

n = 3
top_n = feature_array[tfidf_sorting][:n]

In [170]:
top_n

array(['8成', '頹勢', '扭轉'], dtype=object)